## 1. Execute full pipeline

In [1]:
import random
import time

from keras_tuner import RandomSearch

import numpy as np

import pandas as pd

import tensorflow as tf

from config.constants import (
    FORECAST_HORIZON, NB_TRIALS,
    OBSERVATION_WINDOW, SEED, TRAIN_PERC
)

from src.cut_point_detector import CutPointMethod, CutPointModel, get_cut_point_detector
from src.dataset import read_dataset, split_X_y, split_train_test
from src.forecaster import InternalForecaster, TimeSeriesHyperModel
from src.scaler import Scaler
from src.utils import get_error_results

tf.get_logger().setLevel('ERROR')

np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)


In [2]:
print("Available devices:", tf.config.list_physical_devices())
print("Is GPU available?", tf.config.list_physical_devices('GPU'))


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU available? [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
if tf.config.list_physical_devices("GPU"):
    try:
        tf.config.set_visible_devices([], "GPU")  # Hide traditional GPUs if any (like external ones)
        tf.config.experimental.set_memory_growth(tf.config.list_physical_devices("GPU")[0], True)
        print("✅ TensorFlow is using Apple GPU via MPS")
    except Exception as e:
        print("⚠️ Error setting MPS GPU:", e)
else:
    print("❌ No GPU found, running on CPU")

✅ TensorFlow is using Apple GPU via MPS


In [4]:
timestamp = 'validate_pipeline'
dataset_domain_argv = 'UCI'
dataset_argv = 'APPLIANCES_ENERGY'
cut_point_model_argv = 'Window'
cut_point_method_argv = 'L1'

In [5]:
execution_id = f"{timestamp}_{dataset_domain_argv}_{dataset_argv}_{cut_point_model_argv}_{cut_point_method_argv}_{SEED}"
cut_point_model = CutPointModel.from_str(cut_point_model_argv)
cut_point_method = CutPointMethod.from_str(cut_point_method_argv)
cut_point_approach = f"{cut_point_model.value.title()} {cut_point_method.value.title()}"

In [6]:
print(f"[Step 1] Reading dataset {dataset_argv} from {dataset_domain_argv}")
df, variables = read_dataset(dataset_domain_argv, dataset_argv)
print(f"Variables: {variables}")

[Step 1] Reading dataset APPLIANCES_ENERGY from UCI
Variables: ['T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint']


In [7]:
print("[Step 2] Splitting data into train_val and test")
train_val, test = split_train_test(df)
report = {
    'execution_id': execution_id,
    'timestamp': timestamp,
    'cut_point_model': cut_point_model.value,
    'cut_point_method': cut_point_method.value,
    'cut_point_approach': cut_point_approach,
    'seed': SEED,
    'observation_window': OBSERVATION_WINDOW,
    'train_perc': TRAIN_PERC,
    'nb_trials': NB_TRIALS,
    'dataset_domain': dataset_domain_argv,
    'dataset': dataset_argv,
    'variables': variables,
    'dataset_shape': df.shape,
    'train_val_shape': train_val.shape,
    'test_shape': test.shape,
}

[Step 2] Splitting data into train_val and test


In [8]:
print(f"[Step 3] Detecting cut point ({cut_point_approach})")
start_time = time.time()
cut_point_detector = get_cut_point_detector(cut_point_model, cut_point_method)
cut_point, cut_point_perc = cut_point_detector.find_cut_point(train_val, variables)
end_time = time.time()
detect_cut_point_duration = end_time - start_time
print(f"Cut point: {cut_point}, Cut point percentage: {cut_point_perc}")
report.update({
    'detect_cut_point_duration': detect_cut_point_duration,
    'cut_point': str(cut_point),
    'cut_point_perc': cut_point_perc
})

[Step 3] Detecting cut point (Window L1)
Cut point: 8460, Cut point percentage: 53.5850012667849


In [9]:
print("[Step 4] Reducing train_val based on cut point")
start_time = time.time()
reduced_train_val = cut_point_detector.apply_cut_point(train_val, cut_point)
end_time = time.time()
apply_cut_point_duration = end_time - start_time
report.update({
    'apply_cut_point_duration': apply_cut_point_duration,
    'reduced_train_val.shape': reduced_train_val.shape,
})

[Step 4] Reducing train_val based on cut point


In [10]:
print("[Step 5] Splitting train_val into train and val")
reduced_train, reduced_val = split_train_test(reduced_train_val)
report.update({
    'reduced_train.shape': reduced_train.shape,
    'reduced_val.shape': reduced_val.shape,
})

[Step 5] Splitting train_val into train and val


In [11]:
print("[Step 6] Fitting scaler on train and applying on train and val")
start_time = time.time()
scaler = Scaler(variables)
scaled_reduced_train = scaler.fit_scale(reduced_train)
scaled_reduced_val = scaler.scale(reduced_val)
end_time = time.time()
fit_apply_scaler_train_val_duration = end_time - start_time
report.update({
    'fit_apply_scaler_train_val_duration': fit_apply_scaler_train_val_duration,
})

[Step 6] Fitting scaler on train and applying on train and val


In [12]:
print("[Step 7] Splitting train and val into X and y")
X_reduced_scaled_train, y_reduced_scaled_train = split_X_y(scaled_reduced_train)
X_reduced_scaled_val, y_reduced_scaled_val = split_X_y(scaled_reduced_val)
report.update({
    'X_reduced_scaled_train.shape': X_reduced_scaled_train.shape,
    'y_reduced_scaled_train.shape': y_reduced_scaled_train.shape,
    'X_reduced_scaled_val.shape': X_reduced_scaled_val.shape,
    'y_reduced_scaled_val.shape': y_reduced_scaled_val.shape,
})

[Step 7] Splitting train and val into X and y


In [13]:
y_reduced_scaled_train[0]

array([[ 0.42508344,  0.34656565, -1.55042352, -1.21398052, -1.10125741,
        -0.90743084],
       [ 0.43823785,  0.34907256, -1.60052507, -1.21398052, -1.16634056,
        -0.94966489],
       [ 0.45139227,  0.35157947, -1.65062661, -1.21398052, -1.2314237 ,
        -0.99189893],
       [ 0.46454668,  0.35408636, -1.70072815, -1.21398052, -1.29650684,
        -1.03061347],
       [ 0.47770109,  0.35659326, -1.75082969, -1.21398052, -1.36158999,
        -1.07284751],
       [ 0.4908555 ,  0.36662088, -1.75082969, -1.21398052, -1.34857336,
        -1.04926683],
       [ 0.50400991,  0.37664847, -1.75082969, -1.21398052, -1.33555673,
        -1.02709396]])

In [14]:
y_reduced_scaled_train[1]

array([[ 0.43823785,  0.34907256, -1.60052507, -1.21398052, -1.16634056,
        -0.94966489],
       [ 0.45139227,  0.35157947, -1.65062661, -1.21398052, -1.2314237 ,
        -0.99189893],
       [ 0.46454668,  0.35408636, -1.70072815, -1.21398052, -1.29650684,
        -1.03061347],
       [ 0.47770109,  0.35659326, -1.75082969, -1.21398052, -1.36158999,
        -1.07284751],
       [ 0.4908555 ,  0.36662088, -1.75082969, -1.21398052, -1.34857336,
        -1.04926683],
       [ 0.50400991,  0.37664847, -1.75082969, -1.21398052, -1.33555673,
        -1.02709396],
       [ 0.51716432,  0.38667608, -1.75082969, -1.21398052, -1.3225401 ,
        -1.00245744]])

In [15]:
y_reduced_scaled_train[2]

array([[ 0.45139227,  0.35157947, -1.65062661, -1.21398052, -1.2314237 ,
        -0.99189893],
       [ 0.46454668,  0.35408636, -1.70072815, -1.21398052, -1.29650684,
        -1.03061347],
       [ 0.47770109,  0.35659326, -1.75082969, -1.21398052, -1.36158999,
        -1.07284751],
       [ 0.4908555 ,  0.36662088, -1.75082969, -1.21398052, -1.34857336,
        -1.04926683],
       [ 0.50400991,  0.37664847, -1.75082969, -1.21398052, -1.33555673,
        -1.02709396],
       [ 0.51716432,  0.38667608, -1.75082969, -1.21398052, -1.3225401 ,
        -1.00245744],
       [ 0.53031874,  0.39670369, -1.75082969, -1.21398052, -1.30952347,
        -0.97782091]])

In [16]:
print("[Step 8] Running HPO and NAS")
n_variables = len(variables)
forecaster_hypermodel = TimeSeriesHyperModel(
    n_variables=n_variables
)
forecaster_tuner = RandomSearch(
    forecaster_hypermodel,
    objective='val_loss',
    max_trials=3,
    executions_per_trial=1,
    directory=f"outputs/tuner/delete_me",
    project_name="delete_me",
    seed=SEED,
    overwrite=True,
    distribution_strategy=tf.distribute.MirroredStrategy()
)
start_time = time.time()
forecaster_tuner.search(
    X_reduced_scaled_train,
    y_reduced_scaled_train,
    validation_data=(X_reduced_scaled_val, y_reduced_scaled_val),
    shuffle=False,
)
end_time = time.time()
tuner_duration = end_time - start_time
report.update({
    'tuner_duration': tuner_duration
})

Trial 3 Complete [00h 01m 50s]
val_loss: 0.46575453433584657

Best val_loss So Far: 0.3484160549154407
Total elapsed time: 00h 03m 41s


In [17]:
print("[Step 9] Retrieving best model")
best_trial = forecaster_tuner.oracle.get_best_trials(num_trials=1)[0]
best_forecaster_model = forecaster_tuner.get_best_models(num_models=1)[0]
print(f"Trial ID: {best_trial.trial_id}")
print(f"Hyperparameters: {best_trial.hyperparameters.values}")
print(f"Score: {best_trial.score}")
print("-" * 40)
best_forecaster_model.summary()
best_forecaster_model = InternalForecaster(
    best_forecaster_model,
    len(variables),
    best_trial.hyperparameters.values['batch_size'],
    best_trial.hyperparameters.values['epochs'],
)
report.update({
    'best_trial_id': best_trial.trial_id,
    'best_trial_hyperparameters': best_trial.hyperparameters.values,
    'best_trial_score': best_trial.score,
    'best_forecaster_model': best_forecaster_model.summary(),
})

[Step 9] Retrieving best model
Trial ID: 1
Hyperparameters: {'num_layers': 3, 'units_0': 32, 'learning_rate': 0.0005, 'units_1': 128, 'batch_size': 176, 'epochs': 150, 'units_2': 32}
Score: 0.3484160549154407
----------------------------------------


/Users/zamith/Doutorado/TSForecastingSubsets/env/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 36 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 14, 32)         │         4,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 14, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 14, 128)        │        82,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 32)             │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 42)             │         1,386 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 6)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,186 (430.41 KB)

 Trainable params: 109,802 (428.91 KB)

 Non-trainable params: 384 (1.50 KB)

In [18]:
print("[Step 10] Fitting scaler on train_val and applying on train_val and test")
start_time = time.time()
scaler = Scaler(variables)
scaled_reduced_train_val = scaler.fit_scale(reduced_train_val)
scaled_test = scaler.scale(test)
end_time = time.time()
fit_apply_scaler_train_val_test_duration = end_time - start_time
report.update({
    'fit_apply_scaler_train_val_test_duration': fit_apply_scaler_train_val_test_duration,
})

[Step 10] Fitting scaler on train_val and applying on train_val and test


In [19]:
print("[Step 11] Splitting train_val and test into X and y")
X_reduced_scaled_train_val, y_reduced_scaled_train_val = split_X_y(scaled_reduced_train_val)
X_scaled_test, y_scaled_test = split_X_y(scaled_test)
report.update({
    'X_reduced_scaled_train_val.shape': X_reduced_scaled_train_val.shape,
    'y_reduced_scaled_train_val.shape': y_reduced_scaled_train_val.shape,
    'X_scaled_test.shape': X_scaled_test.shape,
    'y_scaled_test.shape': y_scaled_test.shape,
})

[Step 11] Splitting train_val and test into X and y


In [20]:
print("[Step 12] Retraining best model")
start_time = time.time()
best_forecaster_model.fit(
    X_reduced_scaled_train_val,
    y_reduced_scaled_train_val,
    shuffle=False
)
end_time = time.time()
retrain_duration = end_time - start_time
report.update({
    'retrain_duration': retrain_duration,
})

[Step 12] Retraining best model
Epoch 1/150
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.7673 - learning_rate: 5.0000e-04
Epoch 2/150
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.7382 - learning_rate: 5.0000e-04
Epoch 3/150
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.6894 - learning_rate: 5.0000e-04
Epoch 4/150
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.6752 - learning_rate: 5.0000e-04
Epoch 5/150
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.6939 - learning_rate: 5.0000e-04
Epoch 6/150
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.6695 - learning_rate: 5.0000e-04
Epoch 7/150
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.6474 - learning_rate: 5.0000e-04
Epoch 8/150
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.6072 - learning_rate: 2.5000e-04
Epoch 9/150
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.6196 - learning_rate: 2.5000e-04
Epoch 10/150
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.6341 - learning_rate: 2.5000e-04
Epoch 11/150
41/4

In [21]:
print("[Step 13] Forecasting for test")
start_time = time.time()
y_scaled_pred = best_forecaster_model.forecast(X_scaled_test)
y_scaled_test_flat = y_scaled_test.reshape(-1, n_variables)
y_scaled_pred_flat = y_scaled_pred.reshape(-1, n_variables)
end_time = time.time()
forecasting_test_duration = end_time - start_time
report.update({
    'forecasting_test_duration': forecasting_test_duration,
})

[Step 13] Forecasting for test


2025-03-08 18:24:55.516450: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


109/123 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

2025-03-08 18:24:56.311740: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]


123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [22]:
y_scaled_test_flat

array([[ 0.03078476,  0.40226275,  0.12839626, -1.30151869,  0.16265223,
         0.26020635],
       [ 0.06963033,  0.41015928,  0.06817327, -1.30151869,  0.16265223,
         0.26379162],
       [ 0.1084759 ,  0.41805582,  0.00795029, -1.30151869,  0.16265223,
         0.27096217],
       ...,
       [ 3.86354749, -0.19787384, -1.55784732, -0.04814738, -1.06778434,
         3.46185563],
       [ 3.8117534 , -0.19787384, -1.53375813,  0.03018832, -0.99787317,
         3.4260029 ],
       [ 3.78585635, -0.19787384, -1.50966893,  0.10852403, -0.927962  ,
         3.4260029 ]])

In [23]:
y_scaled_pred_flat

array([[-0.0085964 ,  0.15890133,  0.19826137, -0.9824967 ,  0.06092364,
         0.12453961],
       [-0.03280071,  0.15584467,  0.22010508, -0.8891771 ,  0.03042763,
         0.16099823],
       [-0.02334302,  0.16661575,  0.17401537, -0.8805347 ,  0.01756208,
         0.10404027],
       ...,
       [ 1.1298989 , -0.22508025, -1.1055474 ,  0.2424349 , -0.93742895,
         0.5848468 ],
       [ 1.2322851 , -0.18361491, -1.2810272 ,  0.5104733 , -0.7979611 ,
         0.68309665],
       [ 1.1113065 , -0.01370734, -0.7948964 ,  0.44400248, -0.8914185 ,
         0.61942065]], dtype=float32)

In [24]:
print("[Step 14] Descaling data")
start_time = time.time()
y_test = scaler.descale(pd.DataFrame(y_scaled_test_flat, columns=variables))
y_pred = scaler.descale(pd.DataFrame(y_scaled_pred_flat, columns=variables))
end_time = time.time()
descaling_duration = end_time - start_time
report.update({
    'descaling_duration': descaling_duration,
})

[Step 14] Descaling data


In [25]:
y_test

,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,7.70,759.000000,79.666667,1.000000,40.000000,4.37
1,7.85,759.050000,78.833333,1.000000,40.000000,4.38
2,8.00,759.100000,78.000000,1.000000,40.000000,4.40
3,8.12,759.116667,77.333333,1.000000,37.833333,4.38
4,8.23,759.133333,76.666667,1.000000,35.666667,4.37
...,...,...,...,...,...,...
27484,22.70,755.200000,55.666667,3.333333,23.666667,13.30
27485,22.60,755.200000,56.000000,3.500000,24.500000,13.30
27486,22.50,755.200000,56.333333,3.666667,25.333333,13.30
27487,22.30,755.200000,56.666667,3.833333,26.166667,13.20


In [26]:
y_pred

,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,7.547932,757.459045,80.633423,1.678750,38.787407,3.991600
1,7.454468,757.439697,80.935684,1.877296,38.423897,4.093290
2,7.490989,757.507935,80.297920,1.895684,38.270542,3.934423
3,7.565347,757.400452,80.467232,2.003039,37.988461,4.069316
4,7.312187,757.416870,80.200470,2.126533,37.101505,4.032564
...,...,...,...,...,...,...
27484,12.808443,756.308167,65.127846,4.281259,28.809414,5.048982
27485,12.742531,755.574158,62.340981,4.607983,30.022417,5.841889
27486,11.944168,755.027710,62.592018,4.284908,26.887156,5.275483
27487,12.339527,755.290283,60.163822,4.855186,28.549597,5.549520


In [27]:
print("[Step 15] Calculating evaluation metrics")
total_duration = sum(value for key, value in report.items() if key.endswith('_duration'))
error_results = get_error_results(y_test, y_pred, variables)
print(f"Obtained error results: {error_results}")
report.update({
    'total_duration': total_duration,
    'error_results': error_results,
})

[Step 15] Calculating evaluation metrics
Obtained error results: {'Avg_MAPE': 2304191206888.845, 'Avg_MAE': 3.807246950764673, 'Avg_MSE': 29.41247371110586, 'Avg_RMSE': 4.9041129642840815, 'Avg_R2': 0.37846907115799006, 'Avg_WAPE': 0.02569628205541422, 'T_out_MAPE': 0.23068072838770567, 'T_out_MAE': 3.6974275451358887, 'T_out_MSE': 23.986186839292262, 'T_out_RMSE': 4.897569482844757, 'T_out_R2': 0.024589475233169455, 'T_out_WAPE': 0.27063328827838923, 'Press_mm_hg_MAPE': 0.0053421068800391614, 'Press_mm_hg_MAE': 4.034591287253793, 'Press_mm_hg_MSE': 23.524813320124604, 'Press_mm_hg_RMSE': 4.850238480747581, 'Press_mm_hg_R2': 0.2552111827101041, 'Press_mm_hg_WAPE': 0.005336801232984763, 'RH_out_MAPE': 0.12870674824684916, 'RH_out_MAE': 7.025463759949767, 'RH_out_MSE': 80.88116086735462, 'RH_out_RMSE': 8.993395402591538, 'RH_out_R2': 0.7730534885836967, 'RH_out_WAPE': 0.09844121341536179, 'Windspeed_MAPE': 1230890945494.0793, 'Windspeed_MAE': 0.9476864263956538, 'Windspeed_MSE': 1.355727

In [28]:
report

{'execution_id': 'validate_pipeline_UCI_APPLIANCES_ENERGY_Window_L1_0',
 'timestamp': 'validate_pipeline',
 'cut_point_model': 'Window',
 'cut_point_method': 'L1',
 'cut_point_approach': 'Window L1',
 'seed': 0,
 'observation_window': 14,
 'train_perc': 0.8,
 'nb_trials': 15,
 'dataset_domain': 'UCI',
 'dataset': 'APPLIANCES_ENERGY',
 'variables': ['T_out',
  'Press_mm_hg',
  'RH_out',
  'Windspeed',
  'Visibility',
  'Tdewpoint'],
 'dataset_shape': (19735, 7),
 'train_val_shape': (15788, 7),
 'test_shape': (3947, 7),
 'detect_cut_point_duration': 0.14534878730773926,
 'cut_point': '8460',
 'cut_point_perc': 53.5850012667849,
 'apply_cut_point_duration': 8.320808410644531e-05,
 'reduced_train_val.shape': (7328, 7),
 'reduced_train.shape': (5862, 7),
 'reduced_val.shape': (1466, 7),
 'fit_apply_scaler_train_val_duration': 0.0035452842712402344,
 'X_reduced_scaled_train.shape': (5842, 14, 6),
 'y_reduced_scaled_train.shape': (5842, 7, 6),
 'X_reduced_scaled_val.shape': (1446, 14, 6),
 'y

## 2. What would be the error if we predicted the average values for all variables (Dummy Forecaster)?

In [29]:
X_train_val, y_train_val = split_X_y(train_val)
X_test, y_test = split_X_y(test)

In [30]:
train_val_targets_flat = pd.DataFrame(y_train_val.reshape(-1, len(variables)), columns=variables)
avg_values = train_val_targets_flat.mean(axis=0).to_numpy()

In [31]:
n_test = y_test.shape[0]
dummy_pred = np.tile(avg_values, (n_test, FORECAST_HORIZON, 1))

dummy_pred_flat = dummy_pred.reshape(-1, n_variables)
y_test_flat = pd.DataFrame(y_test.reshape(-1, n_variables), columns=variables)

In [32]:
dummy_error_results = get_error_results(y_test_flat, dummy_pred_flat, variables)
print(f"Error metrics for Dummy Forecaster (predicting average values): \n{dummy_error_results}")

Error metrics for Dummy Forecaster (predicting average values): 
{'Avg_MAPE': 3425021718969.223, 'Avg_MAE': 7.215418809609981, 'Avg_MSE': 118.70230356109563, 'Avg_RMSE': 9.023975290726012, 'Avg_R2': -0.8137139190732777, 'Avg_WAPE': 0.04869908342626568, 'T_out_MAPE': 0.5581505737110357, 'T_out_MAE': 8.04392388105092, 'T_out_MSE': 85.53065629469543, 'T_out_RMSE': 9.24827855845051, 'T_out_R2': -2.4781477730923025, 'T_out_WAPE': 0.5887751805856862, 'Press_mm_hg_MAPE': 0.00621866023244673, 'Press_mm_hg_MAE': 4.703571354779968, 'Press_mm_hg_MSE': 31.910844092516054, 'Press_mm_hg_RMSE': 5.648968409587369, 'Press_mm_hg_R2': -0.01028813733691325, 'Press_mm_hg_WAPE': 0.006221702179580035, 'RH_out_MAPE': 0.32693204736892706, 'RH_out_MAE': 17.103435976028628, 'RH_out_MSE': 465.69433115411005, 'RH_out_RMSE': 21.579952065611963, 'RH_out_R2': -0.30670359708489037, 'RH_out_WAPE': 0.23965435572387742, 'Windspeed_MAPE': 4903590906267.885, 'Windspeed_MAE': 1.742090696100894, 'Windspeed_MSE': 4.1720749928

In [33]:
df_comparison = pd.DataFrame({
    "Trained Model": pd.Series(error_results),
    "Dummy Forecaster": pd.Series(dummy_error_results)
})

df_comparison = df_comparison.round(5)
df_comparison

,Trained Model,Dummy Forecaster
Avg_MAPE,2.304191e+12,3.425022e+12
Avg_MAE,3.807250e+00,7.215420e+00
Avg_MSE,2.941247e+01,1.187023e+02
Avg_RMSE,4.904110e+00,9.023980e+00
Avg_R2,3.784700e-01,-8.137100e-01
Avg_WAPE,2.570000e-02,4.870000e-02
T_out_MAPE,2.306800e-01,5.581500e-01
T_out_MAE,3.697430e+00,8.043920e+00
T_out_MSE,2.398619e+01,8.553066e+01
T_out_RMSE,4.897570e+00,9.248280e+00
